##Loading the Data

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout
from keras.layers import BatchNormalization, Input, Lambda
from keras.layers import Embedding, Flatten, dot, StringLookup
from keras import regularizers
from keras.losses import mse, binary_crossentropy

import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

artist_rankings = 'https://raw.githubusercontent.com/hazelwillows/cs50x/main/artist_rankings.csv'
r = pd.read_csv(artist_rankings)
del r["Unnamed: 0"]
r.head()

,Selena Gomez,BTS,Rihanna,Beyonce,Ariana Grande,Carrie Underwood,Justin Bieber,Miley Cyrus,Conan Gray,NewJeans,...,Harry Styles,Imagine Dragons,J. Cole,John Legend,Kevin Gates,Maroon 5,Michael Jackson,Nicki Minaj,Rod Wave,Young Slo-be
0,NaN,NaN,7.0,10.0,9.0,4.0,7.0,6.0,4.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,9.0,2.0,NaN,9.0,NaN,7.0,5.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,3.0,1.0,3.0,1.0,7.0,8.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,6.0,4.0,7.0,7.0,2.0,NaN,10.0,NaN,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.0,8.0,7.0,9.0,6.0,2.0,NaN,NaN,9.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
songs = 'https://raw.githubusercontent.com/hazelwillows/cs50x/main/songs.csv'
s = pd.read_csv(songs)
s = s.set_index("Unnamed: 0")
s.index.names = ['artist']
s.head()

,1,2,3,4,5,6,7,8,9,10
artist,,,,,,,,,,
Selena Gomez,Calm Down,Ice Cream,Love You Like a Love Song,Same Old Love,Lose You to Love Me,Wolves,The Heart Wants What It Wants,Come & Get It,People You Know,Baila Conmigo
BTS,Run BTS,Dynamite,Permission to Dance,Butter,Yet To Come,MIC Drop,Fire,FAKE LOVE,Boy With Luv,Black Swan
Rihanna,Umbrella,Diamonds,Love on the Brain,Woo,Love the Way You Lie,We Found Love,Stay,Dancing in the Dark,Work,Only Girl
Beyonce,CUFF IT,Halo,Crazy in Love,Partition,Single Ladies,ALIEN SUPERSTAR,BREAK MY SOUL,Beautiful Liar,Run the World,Pretty Hurts
Ariana Grande,positions,Bang Bang,7 Rings,God is a woman,Stuck With U,Side to Side,Into You,Santa Tell Me,no tears left to cry,Boyfriend


In [ ]:
artists = {}
i = 0
for name in list(s.index):
  artists[name] = i
  i += 1

print(artists)

{'Selena Gomez': 0, 'BTS': 1, 'Rihanna': 2, 'Beyonce': 3, 'Ariana Grande': 4, 'Carrie Underwood': 5, 'Justin Bieber': 6, 'Miley Cyrus': 7, 'Conan Gray': 8, 'NewJeans': 9, 'Camila Cabello': 10, 'Nick Jonas': 11, 'Kendrick Lamar': 12, 'Lizzo': 13, 'Doja Cat': 14, 'Sam Smith': 15, 'Adele': 16, 'BLACKPINK': 17, 'AKB48': 18, 'Ty Dolla Sign': 19, 'Bebe Rexha': 20, 'Sakura Gakuin': 21, 'AURORA': 22, 'IU': 23, 'Sabrina Carpenter': 24, 'Bob Marley': 25, 'Drake': 26, 'Fleetwood Mac': 27, 'Frank Ocean': 28, 'Israel Kamakawi': 29, 'Jordan Davis': 30, 'Taylor Swift': 31, 'Khalid': 32, 'Mac Miller': 33, 'The Weekend': 34, 'Lil Baby': 35, 'Lil Durk': 36, 'Post Malone': 37, 'SZA': 38, 'Roddy Rich': 39, 'Shawn Mendes': 40, '21 Savage': 41, '2PAC': 42, 'A boogie wit da hoodie': 43, 'Amine': 44, 'Arctic Monkeys': 45, 'Madonna': 46, 'Aaron May': 47, 'Baby Smoove': 48, 'Brent Faiyaz': 49, 'Capolow': 50, 'Childish Gambino': 51, 'Daniel Caesar': 52, 'Drakeo the Rule': 53, 'Dua Lipa': 54, 'Ed Sheeran': 55, 'H

## Preparing Data

In [ ]:
r.index.names = ['UserID']
r.columns.names = ['artist']
r.head()

artist,Selena Gomez,BTS,Rihanna,Beyonce,Ariana Grande,Carrie Underwood,Justin Bieber,Miley Cyrus,Conan Gray,NewJeans,...,Harry Styles,Imagine Dragons,J. Cole,John Legend,Kevin Gates,Maroon 5,Michael Jackson,Nicki Minaj,Rod Wave,Young Slo-be
UserID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,7.0,10.0,9.0,4.0,7.0,6.0,4.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,9.0,2.0,NaN,9.0,NaN,7.0,5.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,3.0,1.0,3.0,1.0,7.0,8.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,6.0,4.0,7.0,7.0,2.0,NaN,10.0,NaN,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.0,8.0,7.0,9.0,6.0,2.0,NaN,NaN,9.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
r = r.T.unstack().reset_index(level=1, name='ratings').rename(columns={'level_1':'artist'})[['ratings','artist']] # Useful function to turn each column name into its own column with corresponding values from dataset
r = r.reset_index()
r = r[['UserID', 'artist', 'ratings']]
r = r.dropna(how='any',axis=0)
r.head()

,UserID,artist,ratings
2,0,Rihanna,7.0
3,0,Beyonce,10.0
4,0,Ariana Grande,9.0
5,0,Carrie Underwood,4.0
6,0,Justin Bieber,7.0


In [ ]:
print(s.index)

Index(['Selena Gomez', 'BTS', 'Rihanna', 'Beyonce', 'Ariana Grande',
       'Carrie Underwood', 'Justin Bieber', 'Miley Cyrus', 'Conan Gray',
       'NewJeans', 'Camila Cabello', 'Nick Jonas', 'Kendrick Lamar', 'Lizzo',
       'Doja Cat', 'Sam Smith', 'Adele', 'BLACKPINK', 'AKB48', 'Ty Dolla Sign',
       'Bebe Rexha', 'Sakura Gakuin', 'AURORA', 'IU', 'Sabrina Carpenter',
       'Bob Marley', 'Drake', 'Fleetwood Mac', 'Frank Ocean',
       'Israel Kamakawi', 'Jordan Davis', 'Taylor Swift', 'Khalid',
       'Mac Miller', 'The Weekend', 'Lil Baby', 'Lil Durk', 'Post Malone',
       'SZA', 'Roddy Rich', 'Shawn Mendes', '21 Savage', '2PAC',
       'A boogie wit da hoodie', 'Amine', 'Arctic Monkeys', 'Madonna',
       'Aaron May', 'Baby Smoove', 'Brent Faiyaz', 'Capolow',
       'Childish Gambino', 'Daniel Caesar', 'Drakeo the Rule', 'Dua Lipa',
       'Ed Sheeran', 'Halsey', 'Harry Styles', 'Imagine Dragons', 'J. Cole',
       'John Legend', 'Kevin Gates', 'Maroon 5', 'Michael Jackson',
  

In [ ]:
r2 = r.replace(list(s.index), list(artists.values()))
r2.head(10)

,UserID,artist,ratings
2,0,2,7.0
3,0,3,10.0
4,0,4,9.0
5,0,5,4.0
6,0,6,7.0
7,0,7,6.0
8,0,8,4.0
9,0,9,9.0
10,0,10,6.0
11,0,11,8.0


In [ ]:
r.head()

,UserID,artist,ratings
2,0,Rihanna,7.0
3,0,Beyonce,10.0
4,0,Ariana Grande,9.0
5,0,Carrie Underwood,4.0
6,0,Justin Bieber,7.0


# Deep Learning Model

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(r2, test_size=0.10, shuffle=True)
X_validation, X_test = train_test_split(X_test, test_size=0.50, shuffle=True)

In [ ]:
user = Model()
artist = Model()

In [ ]:
r.head()

,UserID,artist,ratings
2,0,Rihanna,7.0
3,0,Beyonce,10.0
4,0,Ariana Grande,9.0
5,0,Carrie Underwood,4.0
6,0,Justin Bieber,7.0


In [ ]:
users = r2['UserID'].unique().tolist()
n_users = len(users)
artists_ids = r2['artist'].unique().tolist()
n_artists = len(artists_ids)

In [ ]:
user_input = Input(shape=[1], dtype='int64')
# user_str = StringLookup(output_mode='int')(user_input)
user_embedding = Embedding(input_dim=len(r2['UserID'].unique().tolist())+1, output_dim=1)(user_input)
user_vec = Flatten()(user_embedding)

artist_input = Input(shape=[1], dtype='int64')
# artist_str = StringLookup(output_mode='int')(artist_input)
artist_embedding = Embedding(input_dim=len(r2['artist'].unique().tolist())+1, output_dim=1)(artist_input)
artist_vec = Flatten()(artist_embedding)

In [ ]:
product = dot([user_vec, artist_vec], axes=1)
model = Model(inputs=[user_input, artist_input], outputs=product)
model.compile('adam', 'mean_absolute_error', run_eagerly=True)

In [ ]:
X_train.dtypes

UserID       int64
artist       int64
ratings    float64
dtype: object

In [ ]:
# from keras import backend as K
# sess = tf.compat.v1.Session()
# K.set_session(sess)
from keras.callbacks import EarlyStopping

early_stopping_monitor = EarlyStopping(monitor="loss", patience=10)

model1 = model.fit(x=[X_train.UserID, X_train.artist],
                    y=X_train.ratings, epochs=200, callbacks=[early_stopping_monitor])

Epoch 1/200
 2/14 [===>..........................] - ETA: 5s - loss: 5.6249 

 3/14 [=====>........................] - ETA: 7s - loss: 5.6145

14/14 [==============================] - 9s 510ms/step - loss: 5.6516
Epoch 2/200
14/14 [==============================] - 2s 127ms/step - loss: 5.6515
Epoch 3/200
14/14 [==============================] - 3s 214ms/step - loss: 5.6513
Epoch 4/200
14/14 [==============================] - 1s 40ms/step - loss: 5.6511
Epoch 5/200
14/14 [==============================] - 2s 116ms/step - loss: 5.6506
Epoch 6/200
14/14 [==============================] - 2s 123ms/step - loss: 5.6498
Epoch 7/200
14/14 [==============================] - 2s 128ms/step - loss: 5.6486
Epoch 8/200
14/14 [==============================] - 0s 28ms/step - loss: 5.6470
Epoch 9/200
14/14 [==============================] - 0s 27ms/step - loss: 5.6448
Epoch 10/200
14/14 [==============================] - 0s 22ms/step - loss: 5.6420
Epoch 11/200
14/14 [==============================] - 0s 21ms/step - loss: 5.6387
Epoch 12/200
14/14 [==============================] - 0s 21ms/step - loss: 5.6347
Epoch 13/200
14/14 [===========

In [ ]:
type(model)

keras.engine.functional.Functional

In [ ]:
artists = {}
i = 0
for name in list(s.index):
  artists[name] = i
  i += 1

print(artists)
artists_id = list(artists.values())
print(artists_id)
artists_names = list(artists.keys())
print(artists_names)

{'Selena Gomez': 0, 'BTS': 1, 'Rihanna': 2, 'Beyonce': 3, 'Ariana Grande': 4, 'Carrie Underwood': 5, 'Justin Bieber': 6, 'Miley Cyrus': 7, 'Conan Gray': 8, 'NewJeans': 9, 'Camila Cabello': 10, 'Nick Jonas': 11, 'Kendrick Lamar': 12, 'Lizzo': 13, 'Doja Cat': 14, 'Sam Smith': 15, 'Adele': 16, 'BLACKPINK': 17, 'AKB48': 18, 'Ty Dolla Sign': 19, 'Bebe Rexha': 20, 'Sakura Gakuin': 21, 'AURORA': 22, 'IU': 23, 'Sabrina Carpenter': 24, 'Bob Marley': 25, 'Drake': 26, 'Fleetwood Mac': 27, 'Frank Ocean': 28, 'Israel Kamakawi': 29, 'Jordan Davis': 30, 'Taylor Swift': 31, 'Khalid': 32, 'Mac Miller': 33, 'The Weekend': 34, 'Lil Baby': 35, 'Lil Durk': 36, 'Post Malone': 37, 'SZA': 38, 'Roddy Rich': 39, 'Shawn Mendes': 40, '21 Savage': 41, '2PAC': 42, 'A boogie wit da hoodie': 43, 'Amine': 44, 'Arctic Monkeys': 45, 'Madonna': 46, 'Aaron May': 47, 'Baby Smoove': 48, 'Brent Faiyaz': 49, 'Capolow': 50, 'Childish Gambino': 51, 'Daniel Caesar': 52, 'Drakeo the Rule': 53, 'Dua Lipa': 54, 'Ed Sheeran': 55, 'H

In [ ]:
artists2 = {}
i = 0
for name in list(s.index):
  artists2[i] = name
  i += 1

print(artists2)

{0: 'Selena Gomez', 1: 'BTS', 2: 'Rihanna', 3: 'Beyonce', 4: 'Ariana Grande', 5: 'Carrie Underwood', 6: 'Justin Bieber', 7: 'Miley Cyrus', 8: 'Conan Gray', 9: 'NewJeans', 10: 'Camila Cabello', 11: 'Nick Jonas', 12: 'Kendrick Lamar', 13: 'Lizzo', 14: 'Doja Cat', 15: 'Sam Smith', 16: 'Adele', 17: 'BLACKPINK', 18: 'AKB48', 19: 'Ty Dolla Sign', 20: 'Bebe Rexha', 21: 'Sakura Gakuin', 22: 'AURORA', 23: 'IU', 24: 'Sabrina Carpenter', 25: 'Bob Marley', 26: 'Drake', 27: 'Fleetwood Mac', 28: 'Frank Ocean', 29: 'Israel Kamakawi', 30: 'Jordan Davis', 31: 'Taylor Swift', 32: 'Khalid', 33: 'Mac Miller', 34: 'The Weekend', 35: 'Lil Baby', 36: 'Lil Durk', 37: 'Post Malone', 38: 'SZA', 39: 'Roddy Rich', 40: 'Shawn Mendes', 41: '21 Savage', 42: '2PAC', 43: 'A boogie wit da hoodie', 44: 'Amine', 45: 'Arctic Monkeys', 46: 'Madonna', 47: 'Aaron May', 48: 'Baby Smoove', 49: 'Brent Faiyaz', 50: 'Capolow', 51: 'Childish Gambino', 52: 'Daniel Caesar', 53: 'Drakeo the Rule', 54: 'Dua Lipa', 55: 'Ed Sheeran', 56

In [ ]:
def user_pred(likes, pred_dict, r2=r2):
  r3 = r2.loc[r2['UserID'] != 0].copy()
  index = int(r3.tail(1).index.item())+1
  for i in likes:
    ar = artists[i]
    r3.loc[index] = [0, ar, likes[i]]
    index += 1
  X_train, X_test = train_test_split(r3, test_size=0.20, shuffle=True)
  model1 = model.fit(x=[X_train.UserID, X_train.artist],
                    y=X_train.ratings, epochs=200, callbacks=[early_stopping_monitor], verbose=0)
  for i in list(artists.values()):
    print(prediction(0, i, pred_dict, r3))

def prediction(user_id, artist, predict_dict, r3):
  dim1 = np.array([user_id])
  dim2 = np.array([artist])
  pred = [dim1, dim2]
  p = model.predict(pred, verbose=0).flatten()[0]
  a = artists2[artist] # get artist name from the id passed
  predict_dict[a] = p
  return f'{a}: {p}'

In [ ]:
my_likes = {
    'BLACKPINK': 10.0,
    'Shawn Mendes': 10.0,
    'Selena Gomez': 10.0,
    'Carrie Underwood': 10.0,
    'BTS': 10.0,
    'Camila Cabello': 8.0,
    'Drake': 10.0,
    'Miley Cyrus': 10.0
}

predict = {}

user_pred(my_likes, predict)

Selena Gomez: 5.631424903869629
BTS: 7.180567264556885
Rihanna: 3.6112489700317383
Beyonce: 6.316577911376953
Ariana Grande: 5.628373146057129
Carrie Underwood: 6.688653469085693
Justin Bieber: 7.519956588745117
Miley Cyrus: 8.459893226623535
Conan Gray: 6.361104965209961
NewJeans: 4.783611297607422
Camila Cabello: 8.041460037231445
Nick Jonas: 4.70451545715332
Kendrick Lamar: 6.132256507873535
Lizzo: 5.020298480987549
Doja Cat: 5.395459175109863
Sam Smith: 5.374671459197998
Adele: 5.888898849487305
BLACKPINK: 4.698750972747803
AKB48: 5.0192365646362305
Ty Dolla Sign: 5.625936508178711
Bebe Rexha: 6.701877593994141
Sakura Gakuin: 7.197940349578857
AURORA: 4.7719597816467285
IU: 4.6893720626831055
Sabrina Carpenter: 6.310352802276611
Bob Marley: 5.754105091094971
Drake: 6.732334136962891
Fleetwood Mac: 6.002638339996338
Frank Ocean: 6.121548175811768
Israel Kamakawi: 6.737593650817871
Jordan Davis: 6.729793071746826
Taylor Swift: 2.510498046875
Khalid: 5.887280464172363
Mac Miller: 5.15

In [ ]:
predict_sorted = sorted(predict.items(), key = lambda x:x[1], reverse = True)
print(predict_sorted)

[('2PAC', 8.58481), ('Miley Cyrus', 8.459893), ('J. Cole', 8.36709), ('Camila Cabello', 8.04146), ('Rod Wave', 7.5411987), ('Childish Gambino', 7.5361047), ('Lil Durk', 7.5359645), ('Justin Bieber', 7.5199566), ('Sakura Gakuin', 7.1979403), ('BTS', 7.1805673), ('SZA', 6.918215), ('Arctic Monkeys', 6.8718467), ('Israel Kamakawi', 6.7375937), ('Drake', 6.732334), ('Jordan Davis', 6.729793), ('Brent Faiyaz', 6.7091885), ('Young Slo-be', 6.7037983), ('Bebe Rexha', 6.7018776), ('Capolow', 6.700721), ('Aaron May', 6.697727), ('Nicki Minaj', 6.6972103), ('Baby Smoove', 6.696585), ('Drakeo the Rule', 6.692988), ('Carrie Underwood', 6.6886535), ('Conan Gray', 6.361105), ('Beyonce', 6.316578), ('Sabrina Carpenter', 6.310353), ('Kendrick Lamar', 6.1322565), ('Frank Ocean', 6.121548), ('Fleetwood Mac', 6.0026383), ('The Weekend', 5.98429), ('Adele', 5.888899), ('Khalid', 5.8872805), ('Lil Baby', 5.858442), ('Bob Marley', 5.754105), ('Madonna', 5.659005), ('Selena Gomez', 5.631425), ('Ariana Grande

In [ ]:
recc = pd.DataFrame.from_dict(predict_sorted)
recc.head(10)

,0,1
0,2PAC,8.584810
1,Miley Cyrus,8.459893
2,J. Cole,8.367090
3,Camila Cabello,8.041460
4,Rod Wave,7.541199
5,Childish Gambino,7.536105
6,Lil Durk,7.535964
7,Justin Bieber,7.519957
8,Sakura Gakuin,7.197940
9,BTS,7.180567
